In [1]:
import requests
import pandas as pd


In [32]:
url = "https://restcountries.com/v3.1/all"
# url= "https://restcountries.com/v3.1/independent?status=true"

def tentar_conexao(url):

    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException:
        return None

data = tentar_conexao(url)


In [33]:

if data is not None:
    print("Dados recebidos:")
    print(data[0])
else:
    print("Falha ao acessar dados do site.")



Falha ao acessar dados do site.


In [34]:

if data is not None:
    table1 = pd.DataFrame([{
    "Nome Comum": country["name"]["common"],
    "Nome Oficial": country["name"]["official"],
    "Código Alpha-2": country["cca2"],
    "Região": country["region"],
    "Latitude": country["latlng"][0] if country["latlng"] else None,
    "Longitude": country["latlng"][1] if country["latlng"] else None,
    "Capital": country["capital"][0] if country.get("capital") else None
    } for country in data])
    print(table1)
else:
    print("Falha ao acessar dados do site.")


Falha ao acessar dados do site.


In [35]:
if data is not None:
    table2 = pd.DataFrame([{
    "Nome Comum": country["name"]["common"],
    "População": country["population"],
    "Área": country["area"],
    "Continente": country["continents"][0],
    "País sem litoral": country["landlocked"]
} for country in data])
    print(table2)
else:
    print("Falha ao acessar dados do site.")


Falha ao acessar dados do site.


In [31]:
if data is not None:
    table3 = pd.DataFrame([{
    "Nome Comum": country["name"]["common"],
    "Independente": country.get("independent"),
    "Membro da ONU": country.get("unMember"),
    "Status": country.get("status")
} for country in data])
    print(table3)
else:
    print("Falha ao acessar dados do site.")


Falha ao acessar dados do site.
